In [2]:
import os
import stat
import cv2
import dlib
import json
import shutil
import numpy as np
import pandas as pd
import mediapipe as mp

from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tensorflow.keras.models import Sequential, Model, save_model, load_model

In [3]:
print(mp.__version__)

0.10.14


In [4]:
# Define paths
base_dir = r'D:\PycharmProjects\source project files\collected_test_data'
extracted_frames_dir = os.path.join(base_dir, "extracted_frames")
cropped_frames_dir = os.path.join(base_dir, "cropped_frames")
combined_image_path = os.path.join(base_dir, "combined_frames.png")

os.chmod(base_dir, stat.S_IWUSR | stat.S_IRUSR | stat.S_IXUSR)

**Frame extraction logic**

In [5]:
def extract_frame(frame, output_dir, frame_index):
    frame_filename = os.path.join(output_dir, f"{frame_index:02d}.png")  # Format frame index with leading zeros
    cv2.imwrite(frame_filename, frame)
    return frame_filename

def extract_frames_with_priority_deletion(video_path, output_dir, target_frames=60):
    # Create base directory for storing extracted frames
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    # Handle case when the video has fewer than the target number of frames
    if total_frames < target_frames:
        for i in range(total_frames):
            extract_frame(frames[i], output_dir, i + 1)  # Start frame index from 1

        # Copy the last frame to fill the deficit until the target number is reached
        last_frame = frames[-1]
        for i in range(total_frames, target_frames):
            extract_frame(last_frame, output_dir, i + 1)

        print(f"Copied last frame to fill the deficit for {video_path}.")
        cap.release()
        return

    # Handle case when the video has more than the target number of frames
    if total_frames > target_frames:
        frames_to_delete = total_frames - target_frames
        delete_from_end = int(frames_to_delete * 0.8)  # 80% of frames to delete from the end
        delete_from_start = frames_to_delete - delete_from_end  # 20% from the start

        # Retain the middle portion after deleting the required frames
        frames = frames[delete_from_start:total_frames - delete_from_end]

    # Extract frames after deletion logic or for target-sized videos
    for i in range(len(frames)):
        extract_frame(frames[i], output_dir, i + 1)

    cap.release()
    print(f"Extracted {len(frames)} frames saved at: {output_dir}")

**frames cropping function and parameters**

In [6]:
# Load the detector and predictor (dlib models)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"D:\PycharmProjects\source project files\models\shape_predictor_68_face_landmarks.dat")

# Mouth crop dimensions
LIP_HEIGHT = 80
LIP_WIDTH = 112

def process_frame(frame_file, path, output_path):
    """
    Save a single frame as a .png image in the specified directory.
    Ensures complete image save before moving to the next frame.
    """
    frame_path = os.path.join(path, frame_file)
    try:
        # Load the frame
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Warning: Could not read frame {frame_path}. Skipping.")
            return False

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale image
        faces = detector(gray)

        if not faces:
            return False

        # Only process if a face is detected
        for face in faces:
            landmarks = predictor(gray, face)

            # Extract the mouth region by iterating over the landmarks (48 to 67)
            mouth_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 68)]
            mouth_points_np = np.array(mouth_points)

            # Find the bounding rectangle around the mouth points
            x, y, w, h = cv2.boundingRect(mouth_points_np)

            # Calculate padding to fit the target dimensions
            width_diff = LIP_WIDTH - w
            height_diff = LIP_HEIGHT - h
            pad_left = max(width_diff // 2, 0)
            pad_right = max(width_diff - pad_left, 0)
            pad_top = max(height_diff // 2, 0)
            pad_bottom = max(height_diff - pad_top, 0)

            # Adjust padding to ensure it doesn’t exceed image boundaries
            pad_left = min(pad_left, x)
            pad_right = min(pad_right, frame.shape[1] - (x + w))
            pad_top = min(pad_top, y)
            pad_bottom = min(pad_bottom, frame.shape[0] - (y + h))

            # Crop and resize the mouth region
            lip_frame = frame[y - pad_top:y + h + pad_bottom, x - pad_left:x + w + pad_right]
            lip_frame = cv2.resize(lip_frame, (LIP_WIDTH, LIP_HEIGHT))

            # Save the cropped mouth region to the output directory
            output_frame_path = os.path.join(output_path, frame_file)
            cv2.imwrite(output_frame_path, lip_frame)

            return True  # Exit after processing the first detected face
    except Exception as e:
        print(f"Error processing frame {frame_path}: {e}")
        return False

**combining cropped frames**

In [7]:
def combine_images(input_path, output_path, output_filename="combined_frames.png"):
    # List all frame files and sort them to maintain order
    frame_files = [f for f in os.listdir(input_path) if f.endswith('.png')]
    frame_files.sort()  # Ensure the frames are in order

    # Check that there are exactly 60 frames
    if len(frame_files) != 60:
        print(f"Warning: {input_path} does not contain exactly 60 frames. Skipping.")
        return

    # Load the first image to get dimensions
    first_image = cv2.imread(os.path.join(input_path, frame_files[0]))
    if first_image is None:
        print(f"Error: Could not read {frame_files[0]}.")
        return

    img_height, img_width, channels = first_image.shape

    # Create an empty array for the combined image (10 rows × 6 columns)
    combined_image = np.zeros((img_height * 10, img_width * 6, channels), dtype=np.uint8)

    # Place each frame into the correct position in the combined image
    for idx, frame_file in enumerate(frame_files):
        img = cv2.imread(os.path.join(input_path, frame_file))
        if img is None:
            print(f"Error: Could not read {frame_file}.")
            continue

        row = idx // 6
        col = idx % 6

        # Place the image in the combined image array
        combined_image[row * img_height:(row + 1) * img_height, col * img_width:(col + 1) * img_width] = img

    # Save the combined image
    cv2.imwrite(os.path.join(output_path, output_filename), combined_image)

    print(f"\nCombined image saved at: {os.path.join(output_path, output_filename)}")

**models loading**

In [8]:
# Load the model from the .h5 file
lip_model = load_model(r'D:\PycharmProjects\source project files\Saved models\lip detection\model2811_361_21_d130_GOOD.h5')
# lip_model.summary()

# Load the saved class labels
with open(r'D:\PycharmProjects\source project files\Saved models\class_labels_cl10.json', 'r') as f:
    class_labels = json.load(f)

In [9]:
# class_labels

In [10]:
# Load T5 model and tokenizer for sentence generation
load_directory = r"D:\PycharmProjects\source project files\Saved models\text generation\t5_fine_tuned_local"
tokenizer = T5Tokenizer.from_pretrained(load_directory)
txt_model = T5ForConditionalGeneration.from_pretrained(load_directory)

**integration with web app**

In [14]:
from flask import Flask, render_template, jsonify
from flask_ngrok import run_with_ngrok
import cv2
import mediapipe as mp
import os
import threading
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import numpy as np

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)

# Directories
os.makedirs(base_dir, exist_ok=True)
os.makedirs(extracted_frames_dir, exist_ok=True)
os.makedirs(cropped_frames_dir, exist_ok=True)

# Initialize MediaPipe Hands and Drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def is_open_hand(hand_landmarks):
    """ Check if all fingers are extended (open hand) """
    for finger_tip, finger_pip in [
        (mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.INDEX_FINGER_PIP),
        (mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP),
        (mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_PIP),
        (mp_hands.HandLandmark.PINKY_TIP, mp_hands.HandLandmark.PINKY_PIP)
    ]:
        if hand_landmarks.landmark[finger_tip].y > hand_landmarks.landmark[finger_pip].y:
            return False  # A finger is not extended
    return True  # All fingers are extended

def is_closed_fist(hand_landmarks):
    """ Check if all fingers are folded (closed fist) """
    for finger_tip, finger_pip in [
        (mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.INDEX_FINGER_PIP),
        (mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP),
        (mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_PIP),
        (mp_hands.HandLandmark.PINKY_TIP, mp_hands.HandLandmark.PINKY_PIP)
    ]:
        if hand_landmarks.landmark[finger_tip].y < hand_landmarks.landmark[finger_pip].y:
            return False  # A finger is extended
    return True  # All fingers are folded

# Initialize variables
recording = False
video_writer = None
executor = ThreadPoolExecutor(max_workers=1)
status_messages = []  # List to store real-time status messages
status_lock = threading.Lock()  # Lock for thread-safe updates

@app.route('/')
def index():
    """Serve the main web interface."""
    return render_template('index.html')

@app.route('/get-status', methods=['GET'])
def get_status():
    """Serve the latest status updates to the web app."""
    with status_lock:
        return jsonify({"status": status_messages})

def add_status_message(message):
    """Add a status message safely."""
    with status_lock:
        status_messages.append(message)
        # Limit the size of status_messages to avoid overflow
        if len(status_messages) > 50:
            status_messages.pop(0)

def process_video_in_background(video_path, extracted_frames_dir, cropped_frames_dir, combined_frames_dir, target_frames=60):
    """Background process to handle frame extraction, cropping, and prediction."""
    try:
        add_status_message("Processing video...")

        # Extract frames
        extract_frames_with_priority_deletion(video_path, extracted_frames_dir, target_frames)
        add_status_message(f"Extracted frames saved at: {extracted_frames_dir}")

        # Crop frames
        frame_files = [f for f in os.listdir(extracted_frames_dir) if f.endswith('.png')]
        cropped_count = sum(process_frame(frame_file, extracted_frames_dir, cropped_frames_dir) for frame_file in frame_files)
        add_status_message(f"Finished processing all frames. Images cropped -> {cropped_count}")

        # Combine frames
        combine_images(cropped_frames_dir, combined_frames_dir)
        add_status_message(f"Combined image saved at: {combined_frames_dir}")

        # Predict word and generate a sentence
        if os.path.exists(combined_image_path):
            img = Image.open(combined_image_path).resize((224, 224), Image.LANCZOS)
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            predictions = lip_model.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)[0]
            predicted_word = class_labels[str(predicted_class_index)]

            input_text = f"Generate a sentence for {predicted_word}:"
            input_ids = tokenizer(input_text, return_tensors="pt").input_ids
            outputs = txt_model.generate(input_ids, max_length=20, do_sample=True, top_k=50, top_p=0.9, temperature=0.9, num_return_sequences=1)
            generated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

            add_status_message(f"Predicted Word: <b><i>{predicted_word}</i></b>")
            add_status_message(f"Generated Sentence: <b><i>{generated_sentence}</i></b>")
            add_status_message("\n")
        else:
            add_status_message("Error: Combined image not found.")

    except Exception as e:
        add_status_message(f"Error in processing: {e}")

def open_video_feed():
    """Open the video feed and detect gestures."""
    global recording, video_writer

    # cap = cv2.VideoCapture(0)

    # Initialize video capture
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    # Set the resolution to the maximum supported by your camera
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Set width (e.g., 1280 for 720p)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # Set height (e.g., 720 for 720p)

    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image_height, image_width, _ = frame.shape
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = cv2.flip(image, 1)
            results = hands.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                    )

                    if is_open_hand(hand_landmarks):
                        cv2.putText(image, "Open Hand Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        if not recording:
                            recording = True
                            video_name = "recording.mp4"
                            output_file = os.path.join(base_dir, video_name)
                            video_writer = cv2.VideoWriter(
                                output_file,
                                cv2.VideoWriter_fourcc(*'mp4v'),
                                30,
                                (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
                            )
                            add_status_message("Recording started...")

                    elif is_closed_fist(hand_landmarks):
                        cv2.putText(image, "Closed Fist Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        if recording:
                            recording = False
                            video_writer.release()
                            add_status_message(f"Recording stopped and saved at: {output_file}")
                            executor.submit(process_video_in_background, output_file, extracted_frames_dir, cropped_frames_dir, base_dir)

            if recording and video_writer is not None:
                video_writer.write(frame)

            cv2.imshow('Hand Gesture Recognition', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    threading.Thread(target=open_video_feed, daemon=True).start()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-646:
Traceback (most recent call last):
  File "D:\PycharmProjects\pro_dis_2\venv\lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "D:\PycharmProjects\pro_dis_2\venv\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "D:\PycharmProjects\pro_dis_2\venv\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "D:\PycharmProjects\pro_dis_2\venv\lib\site-packages\urllib3\connectionpool.py"

Extracted 60 frames saved at: D:\PycharmProjects\source project files\collected_test_data\extracted_frames


INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:37] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:38] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:39] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:40] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:41] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:42] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:43] "GET /get-status HTTP/1.1" 200 -



Combined image saved at: D:\PycharmProjects\source project files\collected_test_data\combined_frames.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:44] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:45] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:46] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:47] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:48] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:49] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:50] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:51] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:52] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:53] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:54] "GET /get-status HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 13:37:55] "GET /get-status HTTP/1.1

2phmTJTFYJ1QAu5VW3q1wHkpAqF_5JYYY2u2XnXB9ZaUPoZtV

In [ ]:
cap.release()